In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

**Vamos importar um dataset de tweets sobre o chatgpt, que pode ser encontrado em: https://www.kaggle.com/datasets/edomingo/chatgpt-1000-daily-tweets**


In [ ]:
df = pd.read_csv('chatgpt_daily_tweets.csv')

In [ ]:
df.head()

**A partir das primeiras linhas do dataframe é possível perceber que ele possui uma grande variedade de informações sobre cada tweet, mas a maioria não será utilizada em nossa análise. Então, vamos começar filtrando as informações relevantes**

In [ ]:
atributos_drop = ['tweet_extracted', 'tweet_id', 'user_description', 'user_created', 'user_followers_count', 'user_following_count', 
                     'source', 'user_tweet_count', 'user_verified', 'user_id', 'user_name', 'user_username']
df = df.drop(columns=atributos_drop)
df.head()

In [ ]:
df.describe()


**Agora que o dataframe já está limpo, é necessário filtrar os tweets relevantes. Para isso, vamos retirar todos os tweets que não possuem curtidas**


In [ ]:
print(len(df))
df_relevante = df.query('like_count > 0')
print(len(df_relevante))

**Note como a quantidade de tweets diminuiu em quase 10 vezes. Isso se deve ao fato de que boa parte dos tweets são criados por bots, que pretendemos ignorar**


**Uma informação que é de grande utilidade é o intervalo de datas que o dataset abrange. Vamos conferir quais são as datas do primeiro e do último tweet:**


In [ ]:
print(df.iloc[0]['tweet_created'])
print(df.iloc[-1]['tweet_created'])

**Também é interessante tentar entender como se comportam as curtidas no nosso conjunto de tweets**

In [ ]:
df_relevante['like_count_log'] = np.log(df_relevante['like_count'])

df_relevante['like_count_log'].hist(bins=9, alpha=1)
plt.title('Distribuição')
plt.xlabel('Ordem de magnitude dos likes')
plt.ylabel('Frequência')
plt.show()

**Agora, vamos entender qual é a distribuição das linguas dos tweets no dataframe filtrado**

In [ ]:
df_linguas = df_relevante.groupby('lang').count()[['text']].reset_index().rename(columns={'text': 'quantidade'})
df_linguas = df_linguas.sort_values('quantidade', ascending = False)
df_linguas = df_linguas.query('quantidade >= 50')
#ordena as quantidades e tira as linguas com menos de 50 tweets

In [ ]:
plt.bar(df_linguas['lang'], df_linguas['quantidade'])
plt.xlabel('Lingua')
plt.ylabel('quantiadade')
plt.title('Quantidade por lingua')
plt.show()

**No nosso trabalho pretendemos utilizar um algoritmo para medir a emoção associada a cada tweet sobre o chatGPT. Entretanto, o algoritmo utilizado não é capaz de fazer essa análise para linguas diferentes do inglês, então é necessário traduzi todos os tweets para o inglês antes de fazer a análise**
**Vamos utilizar apenas os 100 primeiros tweets já que a tradução utuliza uma api do google, então traduzir todas as linhas leva muito tempo**

In [ ]:
def traduzir(tweet):
    return GoogleTranslator(source='auto', target='en').translate(tweet)

In [ ]:
df_100 = df_relevante.head(100)
df_100['traduzido'] = df_100['text'].apply(traduzir)
df_100.head()

**Agora que temos uma coluna traduzida, podemos criar colunas para as análises de sentimento (negativo, positivo e neutro)**

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def analisar(tweet, tipo):
    result = analyzer.polarity_scores(tweet)
    return result[tipo]

In [ ]:
df_100['neg'] = df_100['traduzido'].apply(lambda x: analisar(x, 'neg'))
df_100['pos'] = df_100['traduzido'].apply(lambda x: analisar(x, 'pos'))
df_100['neu'] = df_100['traduzido'].apply(lambda x: analisar(x, 'neu'))
df_100.head()

**Esses sentimentos serão explorados na versão final do projeto. O foco é entender as variações dos sentimentos em relação ao chatGPT à medida que notícias relacionadas à tecnologia eram distribuidas**